#  Data Munchers

Hotel reviews project.
Data: https://www.kaggle.com/jiashenliu/515k-hotel-reviews-data-in-europe

In [1]:
import pandas as pd

users = pd.read_csv('../data/Hotel_Reviews.csv')

users.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [2]:
#Creating new dataframe with reviews columns cobined into one as well as cleaning 'No Negative / Positive' entries.
import random

#remove all other columns
def dropCols(newDF):
    newDF['Reviewer_Score'].describe()
    newDF = newDF.drop(["Hotel_Address", "Additional_Number_of_Scoring", "Review_Date", "Average_Score", 
                        "Reviewer_Nationality", "Review_Total_Negative_Word_Counts", "Total_Number_of_Reviews",
                       "Review_Total_Positive_Word_Counts", "Total_Number_of_Reviews_Reviewer_Has_Given",
                       "Tags", "days_since_review", "lng", "lat", "Hotel_Name"], axis=1)
    return newDF

#reduce the size of the data to our liking
#It randomly chooses rows from the entire dataset to reduce the bias
#rng is the parameter of how many entries we want in our dataset
def reduceDataSize(newDF, rng):    
#     nums = random.sample(range(0, 515737), rng)
    total_rows = newDF.shape[0]
    score_column_index = newDF.columns.get_loc('Reviewer_Score')
    positive = 0
    negative = 0
    d = pd.DataFrame()
    while (positive + negative) < rng:
        row_index = random.sample(range(0, total_rows), 1)
        row = newDF.iloc[row_index, :]
        review_score = row.iloc[:, score_column_index].values[0]
        if (positive <= rng/2) & (review_score > 5):
            d = d.append(row)
            positive += 1
        elif (negative <= rng/2) & (review_score <= 5):
            d = d.append(row)
            negative += 1
   
    return d
        
#fix the no negative and no positive entries
def removeNoNeg_NoPos(df):
    df["Negative_Review"] = df["Negative_Review"].replace('No Negative', '')
    df["Positive_Review"] = df["Positive_Review"].replace('No Positive', '')
    return df

#concatinate the positive and negative entries into one review entry
def mergePosAndNeg(newDF):
    newDF["Review"] = newDF["Negative_Review"].map(str) + newDF["Positive_Review"]
    newDF = newDF.drop(["Negative_Review", "Positive_Review"], axis = 1)
    return newDF

#classify review score as good or bad
def classifyScores(newDF):
    scores = []
    for i in newDF["Reviewer_Score"]:
        if i > 7.5:
            #strr = 'good ' + str(i)
            scores.append(1)
            #print(i)
        else:
            #strr = 'bad ' + str(i)
            scores.append(0)
    newDF['Reviewer_Score'] = scores
    return newDF
    
#newDF['Reviewer_Score_Class'] = scores
#newDF['Reviewer_Score'] = scores
#newDF

In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
stop = stopwords.words('english')

def toLowerCase(newDF):
    newDF['Review'] = newDF['Review'].astype(str).str.lower()
    return newDF

def removeStopWords(text):
    new_text = ""
    word_list = text.split()
    for word in word_list:
        if word not in stop:
            new_text = new_text + word + ' '
    return new_text
        
def rootForm(text):
    new_text = ""
    word_list = text.split()
    for word in word_list:
        new_text = new_text + WordNetLemmatizer().lemmatize(word) + ' '
    return new_text

def removeNumbers(text):
    new_text = ""
    word_list = text.split()
    for word in word_list:
        if not any(c.isdigit() for c in word):
            new_text = new_text + word + ' '
    return new_text

def transform(newDF, how):
    newDF['Review'] = newDF['Review'].apply(lambda x: how(x))                               
    return newDF


[nltk_data] Downloading package stopwords to /Users/Ethan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Ethan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
df = dropCols(users) # drops all unused columns
df = reduceDataSize(df, 5000) # reduce the size of the data to our liking. 10 is just an arbitrary number of rows we want
df = removeNoNeg_NoPos(df) # removes the 'no positive' and 'no negative' entries
df = mergePosAndNeg(df) # if we dont want to merge we can just comment this line out
df = classifyScores(df) # classifies the reviews as either 'good' or 'bad'. Can be 1 or 0. Threshhold is 5
df

,Reviewer_Score,Review
116729,1,Was upgraded to a suite amazing rooms
193012,0,the staffs attitude the receptionists were pl...
123141,1,Tiles in the bathroom not sure if the design ...
195933,1,The lighting in the room wasnt great so it wa...
189477,1,More lighting in the corridor More food varie...
440422,1,Nothing The room was lovely staff were lovely...
70976,1,The mini bar items where a bit pricey The ro...
25958,0,Hygiene en one day THE offer for roomservice ...
387212,1,Nothing really A little pricey 5 mins walk f...
220857,1,Friendly staff nice location good breakfast


In [5]:
df = toLowerCase(df) # lower reviews
df = transform(df, removeStopWords) # remove useless words like ‘the’, ‘a’ ,’this’ etc.
df = transform(df, rootForm) # transform every word into their root form
df = transform(df, removeNumbers) # remove numbers
df.head(5)

,Reviewer_Score,Review
116729,1,upgraded suite amazing room
193012,0,staff attitude receptionist pleasant bar staff...
123141,1,tile bathroom sure design dirty comfy bed love...
195933,1,lighting room wasnt great dark certain part ha...
189477,1,lighting corridor food variety comfortable bed


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
reviews = df['Review']
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(reviews)
feature_names = vectorizer.get_feature_names()
#print(feature_names)
X = X.toarray()
X = X.transpose()
X.shape

mx_doc = {}
for num, col in enumerate(feature_names):
    mx_doc[col] = X[num]
#mx_doc
df_final = pd.DataFrame(data= mx_doc)
df_final['Reviewer_Score'] = df['Reviewer_Score'].values
df_final.head(5)

,aa,aafter,abandoned,ability,abit,able,abnormal,abovemichaeldoll,abrasive,abroad,...,yvonna,zen,zero,zip,zita,zone,zoo,zurich,zzstaff,Reviewer_Score
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [8]:
#Save the cleaned data to another csv we can use for our models
df.to_csv("../data/simpler_clean_data.csv", index=False, encoding='utf8')

#Save the cleaned data to another csv we can use for our models
df_final.to_csv("../data/clean_data.csv", index=False, encoding='utf8')